In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler
import pickle

### read the csv data 

In [16]:
data = pd.read_csv("Churn_Modelling.csv")
data.head()


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [17]:
type(data)

pandas.core.frame.DataFrame

### preprocessing the data 

- dropping the irrelevant features(columns)

In [18]:
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


### encoding categorical variables
- turning the columns Geography and and Gender to numercial representations
- we are only encoding Gender column here to (0,1) values using labelencoder 

In [19]:
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


### using one hot encoding over Geograph column
- because if we give values (0,1,2..) the NN will think more the value more it has weightage (if france is 1 and germany is 2 then germany > france ,unwanted weightage)

In [20]:
from sklearn.preprocessing import OneHotEncoder

one_hot_geographs=OneHotEncoder()
geo_encoded=one_hot_geographs.fit_transform(data[['Geography']])
# type(geo_encoded)
# geo_encoded.toarray()
geo_encoded_df=pd.DataFrame(geo_encoded.toarray(),columns=one_hot_geographs.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [21]:
org_data_nogeo=data.drop('Geography',axis=1)
data=pd.concat([org_data_nogeo,geo_encoded_df],axis=1)
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


### save to pickle file 
- Pickle is a Python library used to serialize (save) Python objects into a file so they can be loaded later.

- 'wb' = write binary mode (required for pickle; also cross-platform safe).
- dump: This function writes the object to the file. It’s like packing the object into a box (the file).

- label_encoder_gender: This is the object being saved. a LabelEncoder object from scikit-learn that has been fitted to encode gender categories (e.g., Male = 0, Female = 1).

In [22]:
with open('label_encode_gender.pkl', 'wb')as file:
    pickle.dump(label_encoder_gender,file)


with open('label_encode_geography.pkl','wb')as file:
    pickle.dump(one_hot_geographs,file)


### dividing the data into independent and dependent features 

In [23]:
X=data.drop(['Exited'],axis=1)
y=data['Exited']

### train test splitting for training and scaling of the values


In [24]:
X_train,X_test,y_train,y_test = train_test_split(X,y,stratify=y,test_size=0.2,random_state=42)

scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)

In [25]:
X_train


array([[ 1.058568  ,  0.90750738,  1.71508648, ...,  1.00175153,
        -0.57831252, -0.57773517],
       [ 0.91362605,  0.90750738, -0.65993547, ..., -0.99825153,
         1.72916886, -0.57773517],
       [ 1.07927399, -1.10191942, -0.18493108, ..., -0.99825153,
         1.72916886, -0.57773517],
       ...,
       [ 0.16821031, -1.10191942, -0.18493108, ...,  1.00175153,
        -0.57831252, -0.57773517],
       [ 0.37527024,  0.90750738, -0.37493284, ..., -0.99825153,
        -0.57831252,  1.73089688],
       [ 1.56586482,  0.90750738,  1.14508121, ..., -0.99825153,
        -0.57831252,  1.73089688]], shape=(8000, 12))

In [26]:
X_test

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
5702,585,1,36,7,0.00,2,1,0,94283.09,1.0,0.0,0.0
3667,525,1,33,4,131023.76,2,0,0,55072.93,0.0,1.0,0.0
1617,557,0,40,4,0.00,2,0,1,105433.53,0.0,0.0,1.0
5673,639,1,34,5,139393.19,2,0,0,33950.08,0.0,0.0,1.0
4272,640,0,34,3,77826.80,1,1,1,168544.85,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5768,674,1,36,2,154525.70,1,0,1,27468.72,1.0,0.0,0.0
833,634,1,37,3,111432.77,2,1,1,167032.49,0.0,1.0,0.0
9719,516,0,47,9,128298.74,1,0,0,149614.17,0.0,1.0,0.0
2776,689,1,39,7,0.00,2,0,0,14917.09,1.0,0.0,0.0


### pickling the scaler and the X_test scaled and y_test


In [27]:
with open('scaler.pkl' , 'wb') as f:
    pickle.dump(scaler,f)


with open('X_test_scaled.pkl', 'wb') as f:
    pickle.dump(X_test_scaled, f)
    
with open('y_test.pkl', 'wb') as f:
    pickle.dump(y_test, f)

### implementing the ANN 

In [28]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [29]:
X_train.shape[1]

12

## IMPLEMENTING THE ANN MODEL

In [ ]:
##building the ANN

model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),#HL1 connected with input layer
    Dense(32,activation='relu'),#HL2 
    Dense(1,activation='sigmoid')#Output layer with sigmoid
])

In [31]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

### compiling the model

In [32]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()

model.compile(optimizer=opt,loss=loss,metrics=['accuracy'])

### setting up tensorboard for visualization of model

In [33]:
log_dir="tensoboard_logs/churn_model/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

### setting up early stopping callback 

In [34]:
earlystop_callback=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

### training the model

In [35]:
history=model.fit(
    X_train,y_train,validation_split=0.2,epochs=100,
    callbacks=[tensorboard_callback,earlystop_callback]
)


#!better than the below method : 
''' history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorboard_callback,earlystop_callback]
) '''



Epoch 1/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8283 - loss: 0.4000 - val_accuracy: 0.8619 - val_loss: 0.3574
Epoch 2/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8481 - loss: 0.3650 - val_accuracy: 0.8669 - val_loss: 0.3375
Epoch 3/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8530 - loss: 0.3520 - val_accuracy: 0.8637 - val_loss: 0.3481
Epoch 4/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8587 - loss: 0.3458 - val_accuracy: 0.8700 - val_loss: 0.3322
Epoch 5/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8559 - loss: 0.3450 - val_accuracy: 0.8725 - val_loss: 0.3268
Epoch 6/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8592 - loss: 0.3407 - val_accuracy: 0.8625 - val_loss: 0.3351
Epoch 7/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8577 - loss: 0.3427 - val_accuracy: 0.8694 - val_loss: 0.3384
Epoch 8/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8595 - loss: 0.3376 - val_accu

' history=model.fit(\n    X_train,y_train,validation_data=(X_test,y_test),epochs=100,\n    callbacks=[tensorboard_callback,earlystop_callback]\n) '

### model saving

In [36]:
model.save('model.h5')

### loading the tensorboard extension
- Without loading the extension, you can’t use the %tensorboard magic command in Jupyter,

In [37]:
%load_ext tensorboard

In [40]:
%tensorboard --logdir tensoboard_logs/churn_model/

Reusing TensorBoard on port 6006 (pid 27028), started 0:00:10 ago. (Use '!kill 27028' to kill it.)